In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import csv

training_examples = []
training_files = [f for f in os.listdir('./training/') if f.endswith('.csv')]
training_files.remove('p108796.csv')  # no dynamic data
training_size = len(training_files)
training_files.sort()
print(training_size)
print(training_files[0])

40335
p000001.csv


In [2]:
static_num = 5
dynamic_num = 34
print(static_num, dynamic_num)

5 34


In [3]:
x = []
static = []
y = []
name = []
mask = []
static_mask = []

for file in training_files:
    x_one = []
    mask_one = []
    static_one = np.zeros((static_num))
    static_mask_one = np.ones((static_num))
    with open(os.path.join('./training/', file), mode='r', encoding='utf-8') as f:
        reader = csv.reader(f)
        los = None
        for row in reader:
            if row[0] == 'HR':
                continue
            if los == None:
                los = int(row[-2])
            x_row = np.zeros((dynamic_num))
            mask_row = np.zeros((dynamic_num))
            for i in range(dynamic_num):
                if row[i] != '':
                    x_row[i] = float(row[i])
                    mask_row[i] = 1
            x_one.append(x_row)
            mask_one.append(mask_row)
        for i in range(static_num):
            if row[i + dynamic_num] != '':
                static_one[i] = float(row[i + dynamic_num])
            else:
                static_mask_one[i] = 0
        assert int(row[-2]) - los + 1 == len(x_one)
    if len(x_one) >= 8:
        x.append(x_one)
        mask.append(mask_one)
        static.append(static_one)
        y.append(int(row[-1]))
        name.append(file.split('.')[0])
        static_mask.append(static_mask_one)
print(len(x), len(mask), len(static), len(y), len(name), len(static_mask))

40335 40335 40335 40335 40335 40335


In [4]:
static_masked = np.ma.masked_array(static, static_mask == 0)
static_mean = np.mean(static_masked, 0)
static_std = np.std(static_masked, 0)
print(static_mean)
print(static_std)

[61.643364819635195 0.559439692574687 0.3041279285979918
 0.30868972356514196 -51.84894508491751]
[16.48294143717642 0.49645435132206406 0.4600370981177364
 0.4619527877721603 139.76471941192648]


In [5]:
x_flat = []
mask_flat = []
for i in range(len(x)):
    x_flat += x[i]
    mask_flat += mask[i]
x_masked = np.ma.masked_array(x_flat, np.array(mask_flat) == 0)
x_mean = np.mean(x_masked, 0)
x_std = np.std(x_masked, 0)
print(x_mean)
print(x_std)

[84.58144338298742 97.19395453339597 36.977228240793444 123.75046539637756
 82.4000998866772 63.830555770342464 18.726497855579872 32.95765667291276
 -0.6899191871174758 24.07548056221939 0.5548386348704794
 7.378934026195934 41.02186880800899 92.65418774854828 260.22338482309493
 23.91545210569777 102.48366144100076 7.557530849328048 105.82790991400108
 1.5106993531749469 1.8361772575251076 136.9322832898959 2.646666023259195
 2.0514502149033444 3.5442376526861317 4.135527970939279
 2.1140594615617503 8.290099451998909 30.794093340027462
 10.430832787915188 41.231193461563606 11.44640501975939 287.3857059168131
 196.01391078961922]
[17.32523590777919 2.936922945242254 0.7700135191976958 23.23154691556354
 16.34174417977269 13.956003531015801 5.098191676423455 7.951593141717585
 4.294271904065938 4.3764700471084845 11.123163961768011
 0.07456736386219906 9.267188230970264 10.892884173242292
 855.7298042255044 19.9942230239091 120.12033815048217 2.4331385255722817
 5.880420206428674 1.8

In [6]:
x_normalize = []
static_normalize = []
mask_normalize = []
for i in range(len(x)):
    static_normalize.append(np.where(static_mask[i] > 0, (static[i] - static_mean) / static_std, 0).tolist())
    x_one = []
    mask_one = []
    for j in range(len(x[i])):
        x_one.append(np.where(mask[i][j] > 0, (x[i][j] - x_mean) / x_std, 0).tolist())
        mask_one.append(np.where(mask[i][j] > 0, 1, 0).tolist())
    x_normalize.append(x_one)
    mask_normalize.append(mask_one)

In [7]:
index_array = list(range(len(x)))
np.random.shuffle(index_array)
print(index_array)

[7108, 27562, 36492, 20595, 37719, 36825, 35016, 28581, 39060, 22736, 5716, 10813, 31421, 19868, 21222, 21346, 29762, 16156, 19173, 12284, 28641, 29965, 16530, 16523, 23963, 3020, 26139, 37855, 15574, 1756, 26096, 5740, 7116, 25786, 30190, 13911, 16754, 7282, 930, 32388, 18519, 9779, 14444, 6328, 23766, 20069, 3474, 37215, 8347, 22336, 9391, 18377, 15355, 14312, 35268, 13599, 25889, 12826, 28760, 29107, 30709, 33122, 29509, 10370, 35011, 26747, 8911, 31946, 23778, 37659, 35356, 14913, 33598, 30077, 37369, 37484, 18613, 11188, 32740, 33619, 14412, 35211, 26044, 15943, 31962, 13628, 8343, 584, 13067, 10101, 34112, 14935, 31451, 17741, 39918, 5414, 5979, 13756, 39541, 38039, 1898, 35673, 24790, 20795, 23720, 32685, 10131, 2620, 31864, 21110, 13400, 17529, 25745, 38568, 3199, 3435, 6286, 29397, 38145, 32571, 29947, 1888, 17003, 27642, 36887, 28376, 23040, 33244, 39217, 35167, 23137, 5301, 16013, 21466, 7206, 21050, 15700, 2088, 37818, 20009, 29035, 26378, 25798, 5622, 29741, 21554, 38873, 

In [8]:
x_random = []
y_random = []
static_random = []
mask_random = []
name_random = []
for i in range(len(index_array)):
    x_random.append(x_normalize[index_array[i]])
    y_random.append(y[index_array[i]])
    static_random.append(static_normalize[index_array[i]])
    mask_random.append(mask_normalize[index_array[i]])
    name_random.append(name[index_array[i]])

In [9]:
pickle.dump((x_random, y_random, static_random, mask_random, name_random), open('data.pkl', 'wb'))
stat_dict = {'x_mean': x_mean, 'x_std': x_std, 'static_mean': static_mean, 'static_std': static_std}
pickle.dump(stat_dict, open('stat.pkl', 'wb'))

In [10]:
cnt = 0
cnt1 = 0
for i in range(len(mask)):
    for j in range(len(mask[i])):
        cnt += sum(mask[i][j])
        cnt1 += len(mask[i][j])
print('Observed Rate:', cnt / cnt1)

Observed Rate: 0.19871573655232633


In [12]:
lens = []
for i in range(len(x)):
    lens.append(len(x[i]))
print(len(x), sum(lens), min(lens), max(lens))

40335 1552160 8 336
